In [1]:
import pandas as pd
import numpy as np
from numpy import average
import matplotlib.pyplot as plt 
import matplotlib.mlab as mlab
import matplotlib.patches as mpatches
import seaborn as sb
import datetime as dt
import glob
import json
from collections import Counter

# called to counteract dark-mode Jupyter from swallowing axes &c.
plt.style.use('seaborn-whitegrid')

In [2]:
# Source citation: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', 
# Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg (http://dx.doi.org/10.1007/s13748-013-0040-3) 
# (accessed: 15 Mar 2022)

# Import the core datasets
daily_core = pd.read_csv('data/uci_data/day.csv')
hourly_core = pd.read_csv('data/uci_data/hour.csv')

<hr>

A fly-by view of the data. Looking for things that poke out.

In [3]:
daily_core.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [4]:
hourly_core.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [5]:
daily_core.head().T

,0,1,2,3,4
instant,1,2,3,4,5
dteday,2011-01-01,2011-01-02,2011-01-03,2011-01-04,2011-01-05
season,1,1,1,1,1
yr,0,0,0,0,0
mnth,1,1,1,1,1
holiday,0,0,0,0,0
weekday,6,0,1,2,3
workingday,0,0,1,1,1
weathersit,2,2,1,1,1
temp,0.344167,0.363478,0.196364,0.2,0.226957


In [6]:
hourly_core.head().T

,0,1,2,3,4
instant,1,2,3,4,5
dteday,2011-01-01,2011-01-01,2011-01-01,2011-01-01,2011-01-01
season,1,1,1,1,1
yr,0,0,0,0,0
mnth,1,1,1,1,1
hr,0,1,2,3,4
holiday,0,0,0,0,0
weekday,6,6,6,6,6
workingday,0,0,0,0,0
weathersit,1,1,1,1,1


In [7]:
daily_core.shape

(731, 16)

In [8]:
hourly_core.shape

(17379, 17)

In [9]:
daily_core.describe().T

,count,mean,std,min,25%,50%,75%,max
instant,731.0,366.000000,211.165812,1.000000,183.500000,366.000000,548.500000,731.000000
season,731.0,2.496580,1.110807,1.000000,2.000000,3.000000,3.000000,4.000000
yr,731.0,0.500684,0.500342,0.000000,0.000000,1.000000,1.000000,1.000000
mnth,731.0,6.519836,3.451913,1.000000,4.000000,7.000000,10.000000,12.000000
holiday,731.0,0.028728,0.167155,0.000000,0.000000,0.000000,0.000000,1.000000
weekday,731.0,2.997264,2.004787,0.000000,1.000000,3.000000,5.000000,6.000000
workingday,731.0,0.683995,0.465233,0.000000,0.000000,1.000000,1.000000,1.000000
weathersit,731.0,1.395349,0.544894,1.000000,1.000000,1.000000,2.000000,3.000000
temp,731.0,0.495385,0.183051,0.059130,0.337083,0.498333,0.655417,0.861667
atemp,731.0,0.474354,0.162961,0.079070,0.337842,0.486733,0.608602,0.840896


In [10]:
hourly_core.describe().T

,count,mean,std,min,25%,50%,75%,max
instant,17379.0,8690.000000,5017.029500,1.00,4345.5000,8690.0000,13034.5000,17379.0000
season,17379.0,2.501640,1.106918,1.00,2.0000,3.0000,3.0000,4.0000
yr,17379.0,0.502561,0.500008,0.00,0.0000,1.0000,1.0000,1.0000
mnth,17379.0,6.537775,3.438776,1.00,4.0000,7.0000,10.0000,12.0000
hr,17379.0,11.546752,6.914405,0.00,6.0000,12.0000,18.0000,23.0000
holiday,17379.0,0.028770,0.167165,0.00,0.0000,0.0000,0.0000,1.0000
weekday,17379.0,3.003683,2.005771,0.00,1.0000,3.0000,5.0000,6.0000
workingday,17379.0,0.682721,0.465431,0.00,0.0000,1.0000,1.0000,1.0000
weathersit,17379.0,1.425283,0.639357,1.00,1.0000,1.0000,2.0000,4.0000
temp,17379.0,0.496987,0.192556,0.02,0.3400,0.5000,0.6600,1.0000


In [11]:
daily_core.nunique(axis=0)

instant       731
dteday        731
season          4
yr              2
mnth           12
holiday         2
weekday         7
workingday      2
weathersit      3
temp          499
atemp         690
hum           595
windspeed     650
casual        606
registered    679
cnt           696
dtype: int64

In [12]:
hourly_core.nunique(axis=0)

instant       17379
dteday          731
season            4
yr                2
mnth             12
hr               24
holiday           2
weekday           7
workingday        2
weathersit        4
temp             50
atemp            65
hum              89
windspeed        30
casual          322
registered      776
cnt             869
dtype: int64

Whoops, what's up with *weathersit*? There is one missing in the daily set.<br />
No really bad weather for a day? Doesn't sound like the D.C. I know.

In [13]:
print(daily_core['weathersit'].unique())
print(hourly_core['weathersit'].unique())

[2 1 3]
[1 2 3 4]


In [14]:
daily_core.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [15]:
hourly_core.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [16]:
daily_core.duplicated().sum()

0

In [17]:
hourly_core.duplicated().sum()

0

In [18]:
daily_core.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [19]:
hourly_core.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

Apparently there are no null values, but there **are** values missing / non-existant.

Can I impute the missing values?<br />
For the missing data in *hourly_core*. The missing hourly entries could be filled. Date & hour can be sequenced, temperature and windspeed filled in from the weather site the UCI researchers used (which by the way takes its data from the weather station at Reagan National Airport located in the middle of Washington D.C. and also supplies NOAA).<br />
The number of bike riders though cannot be imputed because at least one range of missing data covers an extreme weather  event (there is only one entry for 2012-10-29 [Hurricane Sandy]).

In [20]:
# How much missing data is there in the hourly and daily data set?

print("Number of daily entries needed:",((4*30+7*31)*2+28+29), 
        "Number of daily entries missing:", ((4*30+7*31)*2+28+29) - daily_core.shape[0])

# There are no missing days in the daily data set. This makes the calculation of hourly
# entries needed much easier.
# Either hourly entries needed = (4*30+7*31)*2+28+29)*24
# Or as follows

print("Number of hourly entries needed:",(daily_core.shape[0] * 24), 
        "Number of hourly entries missing:", (daily_core.shape[0] * 24 - hourly_core.shape[0]))


Number of daily entries needed: 731 Number of daily entries missing: 0
Number of hourly entries needed: 17544 Number of hourly entries missing: 165


<hr>

Little bit of data maintenance.

In [21]:
# Rename columns
hourly_core.columns = ['instant', 'date_day', 'season', 'year', 'month', 'hour', 'is_holiday', 'weekday', 'is_workday', 'weather', 'temp', 'temp_feel', 'humidity', 'windspeed', 'casual', 'registered', 'total_riders']
daily_core.columns = ['instant', 'date_day', 'season', 'year', 'month', 'is_holiday', 'weekday', 'is_workday', 'weather', 'temp', 'temp_feel', 'humidity', 'windspeed', 'casual', 'registered', 'total_riders']


In [22]:
# Convert time and date to time and date types
hourly_core['hour_time'] = pd.to_datetime(hourly_core['hour'], format='%H').dt.time
hourly_core['date_day'] = pd.to_datetime(hourly_core['date_day']).dt.date

# Combine time and date into an extra column
hourly_core['date_time'] = hourly_core.apply(lambda r : dt.datetime.combine(r['date_day'],r['hour_time']),1)

# Convert date to date types
daily_core['date_day'] = pd.to_datetime(daily_core['date_day']).dt.date

# Combine time and date into an extra column
daily_core['date_time'] = daily_core['date_day']


In [23]:
# Convert certain numerical columns to categories with more human-friendly content
# note: the mapping of seasons is wrong both in the readme file (winter=1) and on the data set website (spring=1), 
# seasons were remapped according to WMO conventions (see note).

hourly_core['season_name'] = hourly_core.month.map({
        1:'winter', 2:'winter', 
        3:'spring', 4:'spring', 5:'spring', 
        6:'summer', 7:'summer', 8:'summer', 
        9:'autumn', 10:'autumn', 11:'autumn', 
        12:'winter'
    }).astype('category')
hourly_core['weekday_name'] = hourly_core.weekday.map({
        1:'monday', 2:'tuesday', 3:'wednesday', 4:'thursday', 5:'friday', 6:'saturday', 0:'sunday'
    }).astype('category')
#hourly_core['is_holiday'] = hourly_core.is_holiday.map({0:'no', 1:'yes'}).astype('category')
hourly_core['weather_desc'] = hourly_core.weather.map({1:'clear', 2:'foggy', 3:'inclement', 4:'crap'}).astype('category')
hourly_core['year'] = hourly_core.year.map({0:2011, 1:2012}).astype('category')

daily_core['season_name'] = daily_core.month.map({
        1:'winter', 2:'winter', 
        3:'spring', 4:'spring', 5:'spring', 
        6:'summer', 7:'summer', 8:'summer', 
        9:'autumn', 10:'autumn', 11:'autumn', 
        12:'winter'}).astype('category')
daily_core['weekday_name'] = daily_core.weekday.map({
        1:'monday', 2:'tuesday', 3:'wednesday', 4:'thursday', 5:'friday', 6:'saturday', 0:'sunday'
    }).astype('category')
#daily_core['is_holiday'] = daily_core.is_holiday.map({0:'no', 1:'yes'}).astype('category')
daily_core['weather_desc'] = daily_core.weather.map({1:'clear', 2:'foggy', 3:'inclement', 4:'crap'}).astype('category')
daily_core['year'] = daily_core.year.map({0:2011, 1:2012}).astype('category')


In [24]:
# Add a time of the day column (morning (06-12), afternoon (12-18), evening (18-00), night (00-06))
hourly_core['daytime'] = hourly_core.hour.apply(lambda value: "night" 
                                            if value <= 6 else "morning"
                                            if value <= 12 else "afternoon"
                                            if value <= 18 else "evening").astype('category')

# This is for the later machine learning section
hourly_core['daytime_code'] = hourly_core.hour.apply(lambda value: 0 
                                            if value <= 6 else 1
                                            if value <= 12 else 2
                                            if value <= 18 else 3).astype('int64')


In [25]:
# Convert time and date to timestamps
hourly_core['hour_time'] = pd.to_datetime(hourly_core['hour'], format='%H').dt.time
hourly_core['date_day'] = pd.to_datetime(hourly_core['date_day']).dt.date

daily_core['date_int'] = daily_core['date_day']
daily_core['date_day'] = pd.to_datetime(daily_core['date_day']).dt.date

# Combine time and date into an extra column
hourly_core['date_time'] = hourly_core.apply(lambda r : dt.datetime.combine(r['date_day'],r['hour_time']),1)
daily_core['date_time'] = daily_core['date_day']


In [26]:
daily_core.dtypes

instant            int64
date_day          object
season             int64
year            category
month              int64
is_holiday         int64
weekday            int64
is_workday         int64
weather            int64
temp             float64
temp_feel        float64
humidity         float64
windspeed        float64
casual             int64
registered         int64
total_riders       int64
date_time         object
season_name     category
weekday_name    category
weather_desc    category
date_int          object
dtype: object

In [27]:
hourly_core.dtypes

instant                  int64
date_day                object
season                   int64
year                  category
month                    int64
hour                     int64
is_holiday               int64
weekday                  int64
is_workday               int64
weather                  int64
temp                   float64
temp_feel              float64
humidity               float64
windspeed              float64
casual                   int64
registered               int64
total_riders             int64
hour_time               object
date_time       datetime64[ns]
season_name           category
weekday_name          category
weather_desc          category
daytime               category
daytime_code             int64
dtype: object

Alternative method to find out how many records are missing, saves you from counting.

In [28]:
# And set the datetime column as index
hourly_core = hourly_core.set_index(['date_time'])
daily_core = daily_core.set_index(['date_time'])

In [29]:
temp = (pd.date_range(start="2011-01-01", end="2012-12-31", freq='1H').difference(hourly_core.index)).to_frame(index=False, name='hr_missing')
print("Number of hourly records missing:", temp.count())

temp = (pd.date_range(start="2011-01-01", end="2012-12-31", freq='1d').difference(daily_core.index)).to_frame(index=False, name='day_missing')
print("Number of daily records missing:", temp.count())

Number of hourly records missing: hr_missing    165
dtype: int64
Number of daily records missing: day_missing    0
dtype: int64


We could drop some unnecessary columns:
* date_time combines yr, mnth, day, hr
* workingday == NOT is_holiday OR saturday/sunday
* instant => date_time index takes over

but they are not really bothering us, and we may be able to use the split date more easily.<br />
Maybe drop them later, if performance is an issue.

df = df.drop(columns=['col1', 'col2'])

In [30]:
# De-normalise the values for temperature, apparent temperature, humidity, 
# and windspeed
# x = x_norm * (max_y - min_y) + min_y
hourly_core['temp'] = hourly_core.temp.apply(lambda x: ((x * (39 - -8)) + -8))
hourly_core['temp_feel'] = hourly_core.temp_feel.apply(lambda x: ((x * (50 - -16)) + -16))
hourly_core['humidity'] = hourly_core.humidity.apply(lambda x: (x * 100))
hourly_core['windspeed'] = hourly_core.windspeed.apply(lambda x: (x * 67))

daily_core['temp'] = daily_core.temp.apply(lambda x: ((x * (39 - -8)) + -8)) # (x*47-8)
daily_core['temp_feel'] = daily_core.temp_feel.apply(lambda x: ((x * (50 - -16)) + -16))
daily_core['humidity'] = daily_core.humidity.apply(lambda x: x * 100)
daily_core['windspeed'] = daily_core.windspeed.apply(lambda x: x * 67)


In [31]:
# Define a function which returns the hours of daylight given the day of the year, from 0 to 365

# def hours_of_daylight(date, axis=23.44, latitude=38.85): # date, axial tilt of earth, latitude of Washington National Airport
#     """Compute the hours of daylight for the given date"""
#     days = (date - 
#         pd.datetime(2011, 1, 1)).days
#     m = (1. - np.tan(np.radians(latitude))
#          * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
#     return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

# daily_core['daylight_hrs'] = list(map(hours_of_daylight, daily_core.index))


In [32]:
hourly_core.dtypes

instant            int64
date_day          object
season             int64
year            category
month              int64
hour               int64
is_holiday         int64
weekday            int64
is_workday         int64
weather            int64
temp             float64
temp_feel        float64
humidity         float64
windspeed        float64
casual             int64
registered         int64
total_riders       int64
hour_time         object
season_name     category
weekday_name    category
weather_desc    category
daytime         category
daytime_code       int64
dtype: object

Note that date_day and hour are still object types and not datetime even though they were converted to timestamps.
Why?

In [33]:
# Dataframe of all missing hourly values
# Datetime as index
missing_hours = pd.date_range(start="2011-01-01", end="2012-12-31", freq='1H').difference(hourly_core.index)
missing_hours = missing_hours.to_frame(index=False, name='hr_missing')

missing_hours['date'] = pd.to_datetime(missing_hours['hr_missing']).dt.date
missing_hours['time'] = pd.to_datetime(missing_hours['hr_missing']).dt.time

missing_hours = missing_hours.set_index(['hr_missing'])

In [34]:
missing_hours.head()

,date,time
hr_missing,,
2011-01-02 05:00:00,2011-01-02,05:00:00
2011-01-03 02:00:00,2011-01-03,02:00:00
2011-01-03 03:00:00,2011-01-03,03:00:00
2011-01-04 03:00:00,2011-01-04,03:00:00
2011-01-05 03:00:00,2011-01-05,03:00:00


In [35]:
# Get the records of a specific date
filtered_df = hourly_core.loc['2011-01-02']
filtered_df

,instant,date_day,season,year,month,hour,is_holiday,weekday,is_workday,weather,...,windspeed,casual,registered,total_riders,hour_time,season_name,weekday_name,weather_desc,daytime,daytime_code
date_time,,,,,,,,,,,,,,,,,,,,,
2011-01-02 00:00:00,25,2011-01-02,1,2011,1,0,0,0,0,2,...,19.9995,4,13,17,00:00:00,winter,sunday,foggy,night,0
2011-01-02 01:00:00,26,2011-01-02,1,2011,1,1,0,0,0,2,...,16.9979,1,16,17,01:00:00,winter,sunday,foggy,night,0
2011-01-02 02:00:00,27,2011-01-02,1,2011,1,2,0,0,0,2,...,19.0012,1,8,9,02:00:00,winter,sunday,foggy,night,0
2011-01-02 03:00:00,28,2011-01-02,1,2011,1,3,0,0,0,2,...,12.9980,2,4,6,03:00:00,winter,sunday,foggy,night,0
2011-01-02 04:00:00,29,2011-01-02,1,2011,1,4,0,0,0,2,...,12.9980,2,1,3,04:00:00,winter,sunday,foggy,night,0
2011-01-02 06:00:00,30,2011-01-02,1,2011,1,6,0,0,0,3,...,19.9995,0,2,2,06:00:00,winter,sunday,inclement,night,0
2011-01-02 07:00:00,31,2011-01-02,1,2011,1,7,0,0,0,2,...,12.9980,0,1,1,07:00:00,winter,sunday,foggy,morning,1
2011-01-02 08:00:00,32,2011-01-02,1,2011,1,8,0,0,0,3,...,15.0013,0,8,8,08:00:00,winter,sunday,inclement,morning,1
2011-01-02 09:00:00,33,2011-01-02,1,2011,1,9,0,0,0,2,...,15.0013,1,19,20,09:00:00,winter,sunday,foggy,morning,1


In [36]:
# Get the records for Hurricane Sandy
# Oct 26: state of emergency declared in the US
# Oct 29-30: all government buildings closed and DC Metro services suspended
filtered_df = hourly_core['2012-10-26':'2012-10-30']
filtered_df.weather.unique()

# For this date range the weather is clear (1), misty (2), and light rain (3).
# Do we agree that a hurricane is at least a 4? Heavy rain and the like?


array([2, 1, 3])

Taking as an example a day where there is only one missing value. If the number of riders (casual, registered, total) is different to the sum of all 23 values present in the hourly data of the day then we can fill in the rider data with the difference and take the weather information from the Washington National Airport weather station.

Result: Checked several days, the difference was always zero. It is unlikely that they all had 0 riders at that time. The sums of the riders in the hourly data from the example day matches the recorded riders from that day's daily record, therefore the daily records were computed using the hourly data with missing records not substituted in any way.

In [37]:
# Example day
filtered_df = hourly_core.loc['2011-01-04']

# Values for that day (2011-01-02): casual: 131, registered: 670, total: 801
# Values for (2011-01-04): 108	1454	1562
print("casuals:", filtered_df.casual.sum())
print("registered:", filtered_df.registered.sum())

casuals: 108
registered: 1454


In [38]:
# Some things to make categories of: cold, hot, and muggy hours and days
# NOAA typically considers relative humidity (RH) levels of 50% or more, and dewpoints 
# (a more direct measure of humidity) above 65 F (18 C) to be uncomfortably high

hourly_core['cold'] = np.where(hourly_core.temp_feel < 5, 1, 0)
hourly_core['hot'] = np.where(hourly_core.temp_feel > 30, 1, 0)
hourly_core['muggy'] = np.where(hourly_core.humidity >= 50, 1, 0)

daily_core['cold'] = np.where(daily_core.temp_feel < 5, 1, 0)
daily_core['hot'] = np.where(daily_core.temp_feel > 30, 1, 0)
daily_core['muggy'] = np.where(daily_core.humidity >= 50, 1, 0)

This came up during training linear regression using the hourly data set:<br />
* 2011-03-10 has humidity = 0 (impossible even in a 'dry room')
* 2012-08-17 has felt temperature around 0°C while measured temperature was around 25°C

In [39]:
# Zero humidity is pretty much impossible on Earth, even a "dry room" has 1%
# This occurs for all humidity records of day 2011-03-10

temp = hourly_core.loc[hourly_core['humidity'] == 0]
temp

# A check on freemeteo, which supplied the weather data for the core data set
# gives the weather as rainy and relative humidity at 100%
hourly_core.loc[hourly_core.humidity == 0, "humidity"] = 100

In [119]:
# Instances where the measured temperature minus 10°C is still larger than the apparent temperature
temp = hourly_core.loc[(hourly_core.temp -10) > hourly_core.temp_feel]
temp

# On 2012-08-17 the apparent temperature was around 0°C while the measured temperature was 20-35°C
# Taken from freemeteo the apparent temperature for this day was:
# felt = [25, 24, 24, 23, 23, 23, 22, 23, 25, 27, 29, 30, 32, 33, 34, 33, 32, 32, 32, 29, 28, 25, 25, 24]

hourly_core.loc[hourly_core.instant == 14132, "temp_feel"] = 25
hourly_core.loc[hourly_core.instant == 14133, "temp_feel"] = 24
hourly_core.loc[hourly_core.instant == 14134, "temp_feel"] = 24
hourly_core.loc[hourly_core.instant == 14135, "temp_feel"] = 23
hourly_core.loc[hourly_core.instant == 14136, "temp_feel"] = 23
hourly_core.loc[hourly_core.instant == 14137, "temp_feel"] = 23
hourly_core.loc[hourly_core.instant == 14138, "temp_feel"] = 22
hourly_core.loc[hourly_core.instant == 14139, "temp_feel"] = 23
hourly_core.loc[hourly_core.instant == 14140, "temp_feel"] = 25
hourly_core.loc[hourly_core.instant == 14141, "temp_feel"] = 27
hourly_core.loc[hourly_core.instant == 14142, "temp_feel"] = 29
hourly_core.loc[hourly_core.instant == 14143, "temp_feel"] = 30
hourly_core.loc[hourly_core.instant == 14144, "temp_feel"] = 32
hourly_core.loc[hourly_core.instant == 14145, "temp_feel"] = 33
hourly_core.loc[hourly_core.instant == 14146, "temp_feel"] = 34
hourly_core.loc[hourly_core.instant == 14147, "temp_feel"] = 33
hourly_core.loc[hourly_core.instant == 14148, "temp_feel"] = 32
hourly_core.loc[hourly_core.instant == 14149, "temp_feel"] = 32
hourly_core.loc[hourly_core.instant == 14150, "temp_feel"] = 32
hourly_core.loc[hourly_core.instant == 14151, "temp_feel"] = 29
hourly_core.loc[hourly_core.instant == 14152, "temp_feel"] = 28
hourly_core.loc[hourly_core.instant == 14153, "temp_feel"] = 25
hourly_core.loc[hourly_core.instant == 14154, "temp_feel"] = 25
hourly_core.loc[hourly_core.instant == 14155, "temp_feel"] = 24

# Now do this with a loop or some clever python function!


The inverse where the measured temperature plus 10°C is less than the apparent temperature<br />
temp = hourly_core.loc[(hourly_core.temp +10) < hourly_core.temp_feel]<br />
does not work as humidity and windchill play a greater role here.

In [120]:
hourly_core.reset_index(inplace=True)
hourly_core.to_csv('data/mod_data/hour_mod.csv', index=False)
daily_core.reset_index(inplace=True)
daily_core.to_csv('data/mod_data/day_mod.csv', index=False)

<hr>

Adding the trip history data for 2011 and 2012.

In [121]:
trips_2011 = pd.read_csv('data/cbs_data/2011-capitalbikeshare-tripdata.csv')

path = r'data/cbs_data/2012'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

trips_2012 = pd.concat(li, axis=0, ignore_index=True)

In [122]:
trips_2011.columns = trips_2011.columns.str.replace(' ','_').str.lower()
trips_2012.columns = trips_2012.columns.str.replace(' ','_').str.lower()


In [123]:
trips_2011.head()

,duration,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual


In [124]:
trips_2012.head()

,duration,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type
0,192,2012-04-01 00:01:13,2012-04-01 00:04:26,31237,25th St & Pennsylvania Ave NW,31212,21st & M St NW,W00663,Member
1,978,2012-04-01 00:01:49,2012-04-01 00:18:08,31225,C & O Canal & Wisconsin Ave NW,31228,8th & H St NW,W00574,Member
2,843,2012-04-01 00:03:25,2012-04-01 00:17:28,31230,Metro Center / 12th & G St NW,31201,15th & P St NW,W00201,Member
3,408,2012-04-01 00:03:37,2012-04-01 00:10:26,31110,20th St & Florida Ave NW,31239,Rhode Island & Connecticut Ave NW,W01060,Casual
4,163,2012-04-01 00:04:13,2012-04-01 00:06:57,31104,Adams Mill & Columbia Rd NW,31112,Harvard St & Adams Mill Rd NW,W01071,Casual


In [125]:
# Add field to each data frame
trips_2011['year'] = '2011'
trips_2012['year'] = '2012'

# and concatenate
bike_trips = pd.concat([trips_2011, trips_2012],axis=0)

# Give some more detail
bike_trips.rename(columns={'duration': 'duration_sec'}, inplace=True)


In [126]:
bike_trips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3255678 entries, 0 to 2028910
Data columns (total 10 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   duration_sec          int64 
 1   start_date            object
 2   end_date              object
 3   start_station_number  int64 
 4   start_station         object
 5   end_station_number    int64 
 6   end_station           object
 7   bike_number           object
 8   member_type           object
 9   year                  object
dtypes: int64(3), object(7)
memory usage: 273.2+ MB


In [127]:
# Some maintenance
bike_trips = bike_trips.astype({'start_date': 'datetime64', 'end_date': 'datetime64', 'member_type': 'category', 'year': 'category'})

bike_trips['date'] = bike_trips['start_date'].dt.date
bike_trips['hour'] = bike_trips['start_date'].dt.hour

# Convert time and date to timestamps
bike_trips['date'] = pd.to_datetime(bike_trips['date']).dt.date
bike_trips['hour'] = pd.to_datetime(bike_trips['hour'], format='%H').dt.time

# Combine time and date into an extra column
bike_trips['date_hour'] = bike_trips.apply(lambda r : dt.datetime.combine(r['date'],r['hour']),1)

# Add weekday
bike_trips['weekday'] = bike_trips['start_date'].dt.dayofweek

# We could drop these but I'm hanging on to them for now
#bike_trips.drop(['date', 'hour'], axis=1, inplace=True)

In [128]:
bike_trips.nunique()

duration_sec              20865
start_date              3092552
end_date                3083001
start_station_number        194
start_station               194
end_station_number          194
end_station                 194
bike_number                1771
member_type                   3
year                          2
date                        731
hour                         24
date_hour                 17379
weekday                       7
dtype: int64

There are 194 stations and 1771 bikes.

<hr>

In [129]:
# There are 21 records where the member type is 'Unknown'
bike_trips.loc[bike_trips.member_type == 'Unknown']

,duration_sec,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,year,date,hour,date_hour,weekday
414633,501,2011-06-09 14:34:39,2011-06-09 14:43:00,31205,21st & I St NW,31232,7th & F St NW / National Portrait Gallery,W00182,Unknown,2011,2011-06-09,14:00:00,2011-06-09 14:00:00,3
424868,2233,2011-06-11 17:40:02,2011-06-11 18:17:15,31215,Georgetown Harbor / 30th St NW,31214,17th & Corcoran St NW,W00812,Unknown,2011,2011-06-11,17:00:00,2011-06-11 17:00:00,5
787755,257,2011-08-31 10:17:31,2011-08-31 10:21:48,31218,L'Enfant Plaza / 7th & C St SW,31219,10th St & Constitution Ave NW,W00239,Unknown,2011,2011-08-31,10:00:00,2011-08-31 10:00:00,2
1059539,443,2011-11-06 01:29:12,2011-11-06 01:36:35,31010,S Glebe & Potomac Ave,31013,Eads & 22nd St S,W01017,Unknown,2011,2011-11-06,01:00:00,2011-11-06 01:00:00,6
112372,1341,2012-04-19 16:23:59,2012-04-19 16:46:20,31014,Lynn & 19th St North,31007,Crystal City Metro / 18th & Bell St,W20053,Unknown,2012,2012-04-19,16:00:00,2012-04-19 16:00:00,3
114829,1039,2012-04-19 20:17:07,2012-04-19 20:34:27,31215,Georgetown Harbor / 30th St NW,31609,Maine Ave & 7th St SW,W01191,Unknown,2012,2012-04-19,20:00:00,2012-04-19 20:00:00,3
118863,384,2012-04-20 14:34:16,2012-04-20 14:40:41,31234,20th & O St NW / Dupont South,31246,M St & Pennsylvania Ave NW,W00491,Unknown,2012,2012-04-20,14:00:00,2012-04-20 14:00:00,4
495782,2242,2012-06-20 00:05:36,2012-06-20 00:42:59,31235,19th St & Constitution Ave NW,31220,US Dept of State / Virginia Ave & 21st St NW,W00703,Unknown,2012,2012-06-20,00:00:00,2012-06-20 00:00:00,2
576117,959,2012-10-02 17:59:28,2012-10-02 18:15:27,31230,Metro Center / 12th & G St NW,31603,1st & M St NE,W01193,Unknown,2012,2012-10-02,17:00:00,2012-10-02 17:00:00,1
693772,577,2012-10-20 03:00:16,2012-10-20 03:09:54,31102,11th & Kenyon St NW,31203,14th & Rhode Island Ave NW,W00106,Unknown,2012,2012-10-20,03:00:00,2012-10-20 03:00:00,5


In [130]:
# Check the 21 records where the member_type is 'Unknown' and see if they are part of the hourly records.

hourly_core[hourly_core.date_time == '2012-11-05 02:00:00'].\
            groupby(['casual', 'registered', 'total_riders']).count()

,,,date_time,instant,date_day,season,year,month,hour,is_holiday,weekday,is_workday,...,windspeed,hour_time,season_name,weekday_name,weather_desc,daytime,daytime_code,cold,hot,muggy
casual,registered,total_riders,,,,,,,,,,,,,,,,,,,,,
2,4,6,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [131]:
bike_trips[bike_trips.date_hour == '2012-11-05 02:00:00'].groupby(['member_type']).count()

# Result: The hourly records disregard the instances of the trip data sets where the member_type is 'Unknown'
# Should probably drop these instances but will keep them in case I map bike routes

,duration_sec,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,year,date,hour,date_hour,weekday
member_type,,,,,,,,,,,,,
Casual,2,2,2,2,2,2,2,2,2,2,2,2,2
Member,4,4,4,4,4,4,4,4,4,4,4,4,4
Unknown,1,1,1,1,1,1,1,1,1,1,1,1,1


In [132]:
bike_trips.head()

,duration_sec,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,year,date,hour,date_hour,weekday
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member,2011,2011-01-01,00:00:00,2011-01-01,5
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual,2011,2011-01-01,00:00:00,2011-01-01,5
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,Georgia & New Hampshire Ave NW,31104,Adams Mill & Columbia Rd NW,W00357,Member,2011,2011-01-01,00:00:00,2011-01-01,5
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,10th & U St NW,31503,Florida Ave & R St NW,W00970,Member,2011,2011-01-01,00:00:00,2011-01-01,5
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,Adams Mill & Columbia Rd NW,31106,Calvert & Biltmore St NW,W00346,Casual,2011,2011-01-01,00:00:00,2011-01-01,5


In [133]:
duplicates = bike_trips[bike_trips.duplicated()]
duplicates

,duration_sec,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,year,date,hour,date_hour,weekday


Nice! No duplicates.

In [134]:
bike_trips.describe().T

,count,mean,std,min,25%,50%,75%,max
duration_sec,3255678.0,1084.497036,2100.266770,60.0,399.0,658.0,1092.0,86355.0
start_station_number,3255678.0,31289.742469,193.121189,31000.0,31200.0,31226.0,31401.0,31806.0
end_station_number,3255678.0,31291.821241,192.376122,31000.0,31201.0,31225.0,31402.0,31806.0
weekday,3255678.0,2.998742,1.974851,0.0,1.0,3.0,5.0,6.0


There is a minimum of 60 seconds for a bike trip. That is a bit short.<br />
Also a maximum of 86355 seconds, that is 45 seconds short of 24 hours.<br />
We'll take a look at that later.

<hr>

Cleaning up the bike trips data.

In [135]:
# Some cosmetic changes. There are 19329 records where the prefix of 
# the bike number is in lowercase.

bike_trips['bike_number'] = bike_trips['bike_number'].str.upper()

In [136]:
# check for incomplete records: no bike number

# Check if the bike number is invalid, i.e. not W+five digits
no_bike = bike_trips[~bike_trips['bike_number'].str.contains('W')]

# There are 4801 records that do not contain valid bike numbers
# The bike number slots are filled by 16 different codes in a 
# 0X-16 digits and letters format

#no_bike.bike_number
no_bike.bike_number.nunique()


16

In [137]:
replace_bike = no_bike.bike_number.unique()
replace_bike

array(['?(0XFFFFFFFFAAC5A4C0)', '?(0X000000003EB026B9)',
       '?(0X000000007C120F6A)', '?(0XFFFFFFFF9A5FEA16)',
       '?(0XFFFFFFFFEBA95C18)', '?(0XFFFFFFFFEE8DADF1)',
       '?(0XFFFFFFFFFAF0B948)', '?(0X0000000026B3BBA8)',
       '?(0X000000002FD1067B)', '?(0X000000004752DD3A)',
       '?(0X000000005F128A5C)', '?(0XFFFFFFFFEB7B5641)',
       '?(0X000000006945F6DC)', '?(0XFFFFFFFF83DE3658)',
       '?(0XFFFFFFFFB44D36A5)', '?(0X000000004443704F)'], dtype=object)

In [138]:
# We could remove the records with no valid bike numbers
# bike_trips.drop(bike_trips[~bike_trips['bike_number'].str.contains('W')].index, inplace=True)

# Or rename the HEX-bikes with a new ID => W+99+xxx

dict = {'?(0XFFFFFFFFAAC5A4C0)':'W99001', '?(0X000000003EB026B9)':'W99002',
       '?(0X000000007C120F6A)':'W99003', '?(0XFFFFFFFF9A5FEA16)':'W99004',
       '?(0XFFFFFFFFEBA95C18)':'W99005', '?(0XFFFFFFFFEE8DADF1)':'W99006',
       '?(0XFFFFFFFFFAF0B948)':'W99007', '?(0X0000000026B3BBA8)':'W99008',
       '?(0X000000002FD1067B)':'W99009', '?(0X000000004752DD3A)':'W99010',
       '?(0X000000005F128A5C)':'W99011', '?(0XFFFFFFFFEB7B5641)':'W99012',
       '?(0X000000006945F6DC)':'W99013', '?(0XFFFFFFFF83DE3658)':'W99014',
       '?(0XFFFFFFFFB44D36A5)':'W99015', '?(0X000000004443704F)':'W99016'}

bike_trips = bike_trips.replace({'bike_number': dict})

In [139]:
no_bike = bike_trips[~bike_trips['bike_number'].str.contains('W')]
no_bike

,duration_sec,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,year,date,hour,date_hour,weekday


In [140]:
# check and remove incomplete records: no member type == Unknown
no_member = bike_trips[bike_trips['member_type'] == 'Unknown']
no_member.count()

# There are 21 records where the member type is unknown. 
# These were also not entered in the daily or hourly record.
bike_trips.drop(bike_trips[bike_trips['member_type'] == 'Unknown'].index, inplace=True)

In [141]:
# Check for non standard start stations = NONE and start station numbers = NONE
#bike_trips.start_station.unique()
#bike_trips.start_station.nunique()

# Check for non standard end stations = NONE and end station numbers = NONE
bike_trips.end_station.unique()
#bike_trips.end_station.nunique()

# There are 194 unique start and end stations. No empty values.

array(['5th & F St NW', '14th & V St NW', 'Adams Mill & Columbia Rd NW',
       'Florida Ave & R St NW', 'Calvert & Biltmore St NW',
       '4th & East Capitol St NE', '15th & P St NW', '14th & R St NW',
       '10th & U St NW', 'S Glebe & Potomac Ave', '7th & T St NW',
       'Georgia & New Hampshire Ave NW', '17th & Corcoran St NW',
       '14th & D St SE', '16th & Harvard St NW',
       'Park Rd & Holmead Pl NW', '10th & Monroe St NE',
       'Lincoln Park / 13th & East Capitol St NE ',
       'New Hampshire Ave & T St NW',
       '14th St Heights / 14th & Crittenden St NW',
       'M St & New Jersey Ave SE', '17th & K St NW',
       '14th & Harvard St NW', 'Georgia Ave and Fairmont St NW',
       '18th & M St NW', '3rd & H St NE', 'Harvard St & Adams Mill Rd NW',
       '19th & L St NW', '14th St & Spring Rd NW', '12th & Army Navy Dr',
       '19th St & Pennsylvania Ave NW', 'Lamont & Mt Pleasant NW',
       '21st & I St NW', '27th & Crystal Dr', 'Van Ness Metro / UDC',
       'Mas

In [142]:
# Check for stalled transactions or users that reconsidered: 
# same start and end station and the duration of ride is 60 seconds or less
# There are 136 records with these specifications

stalled = bike_trips.loc[(bike_trips['duration_sec'] <= 60) & (bike_trips['start_station_number'] == bike_trips['end_station_number'])]

# Remove the 60 second bike ride records
bike_trips.drop(bike_trips[(bike_trips['duration_sec'] <= 60) & (bike_trips['start_station_number'] == bike_trips['end_station_number'])].index, inplace=True)

In [143]:
# Check for bike rides that are 60 seconds or less 
# and have different start and stop stations
# There are 27 of these
bit_moved = bike_trips.loc[(bike_trips['duration_sec'] <= 60) & (bike_trips['start_station_number'] != bike_trips['end_station_number'])]

# Of these all except one lie very close together (customer changed their mind?) 
# and one (ID 507845) is 3.5 miles apart which is impossible
# to drive in 1 minute (12 minutes by car)
# Remove these 27 records

bike_trips.drop(bike_trips[(bike_trips['duration_sec'] <= 60) & (bike_trips['start_station_number'] != bike_trips['end_station_number'])].index, inplace=True)

In [144]:
# Check for bike rides that are over 23 hours. 
# Either the customer forgot to book it back in, the endpoint did not accept the 
# transaction, or something else happened. Gives a fine of $1200, so strong incentive 
# to make sure to book back in.
# There are 6 of these
long_ride = bike_trips.loc[(bike_trips['duration_sec'] >= 86_000)] 

# Remove these 6 records
bike_trips.drop(bike_trips[(bike_trips['duration_sec'] >= 86_000)].index, inplace=True)


In [145]:
bike_trips.to_csv('data/mod_data/bike_trips_2011_2012.csv')

<hr>

In [146]:
# How many unique bikes are there for each year
bike_trips.groupby('year')['bike_number'].nunique()


year
2011    1325
2012    1749
Name: bike_number, dtype: int64

In [147]:
# How many unique start stations are there for each year
print("Start stations:")
print(bike_trips.groupby('year')['start_station_number'].nunique())
print("End stations:")
print(bike_trips.groupby('year')['end_station_number'].nunique())


Start stations:
year
2011    144
2012    191
Name: start_station_number, dtype: int64
End stations:
year
2011    144
2012    191
Name: end_station_number, dtype: int64


So, 424 bikes and 47 stations were added in 2012.<br />
It would be interesting to know which bikes are no longer part of the fleet in 2012 and which ones were added in 2012.<br />
Also, which stations were added in 2012 and which were potentially retired.


In [148]:
# All stations of 2011 and 2012 
# TODO: make this more pythonic
stations_2011 = bike_trips.loc[bike_trips['year']=='2011']
stations_2011 = stations_2011['start_station_number'].unique()
stations_2012 = bike_trips.loc[bike_trips['year']=='2012']
stations_2012 = stations_2012['start_station_number'].unique()


In [149]:
# These are the 3 stations that exist in 2011 but not anymore in 2012
# returns values in stations_2011 not in stations_2012
lost_stations = np.setdiff1d(stations_2011, stations_2012, assume_unique=False)
lost_stations


array([31261, 31266, 31704])

In [150]:
# These are the 50 stations that exist in 2012 but not in 2011
# returns values in stations_2012 not in stations_2011
new_stations = np.setdiff1d(stations_2012, stations_2011, assume_unique=False)
new_stations


array([31023, 31024, 31025, 31026, 31027, 31028, 31029, 31030, 31031,
       31032, 31033, 31034, 31035, 31036, 31037, 31038, 31039, 31040,
       31041, 31042, 31043, 31044, 31045, 31046, 31047, 31048, 31049,
       31050, 31051, 31234, 31240, 31243, 31247, 31248, 31249, 31250,
       31251, 31252, 31253, 31254, 31255, 31256, 31257, 31258, 31306,
       31507, 31508, 31612, 31628, 31803])

In [151]:
# All bikes of 2011 and 2012 
# TODO: make this more pythonic
bikes_2011 = bike_trips.loc[bike_trips['year']=='2011']
bikes_2011 = bikes_2011['bike_number'].unique()
bikes_2012 = bike_trips.loc[bike_trips['year']=='2012']
bikes_2012 = bikes_2012['bike_number'].unique()


In [152]:
lost_bikes = np.setdiff1d(bikes_2011, bikes_2012, assume_unique=False)
lost_bikes


array(['W00073', 'W00257', 'W00320', 'W00457', 'W00758', 'W00779',
       'W00984', 'W01019', 'W01082', 'W99001', 'W99002', 'W99003',
       'W99004', 'W99005', 'W99006', 'W99007', 'W99008', 'W99010',
       'W99011', 'W99012', 'W99013', 'W99015'], dtype=object)

Note that all the HEX-bikes are in the list of "lost" bikes.


In [153]:
new_bikes = np.setdiff1d(bikes_2012, bikes_2011, assume_unique=False)
new_bikes

array(['W00108', 'W00403', 'W00461', 'W00712', 'W00785', 'W00824',
       'W00875', 'W01134', 'W01135', 'W01137', 'W01138', 'W01139',
       'W01140', 'W01141', 'W01142', 'W01143', 'W01144', 'W01145',
       'W01146', 'W01147', 'W01148', 'W01149', 'W01150', 'W01161',
       'W01162', 'W01163', 'W01164', 'W01165', 'W01166', 'W01167',
       'W01168', 'W01169', 'W01170', 'W01194', 'W01196', 'W01216',
       'W01226', 'W01242', 'W01243', 'W01246', 'W01258', 'W01261',
       'W01263', 'W01273', 'W01275', 'W01276', 'W01281', 'W01287',
       'W01289', 'W01296', 'W01308', 'W01317', 'W01324', 'W01334',
       'W01335', 'W01336', 'W01340', 'W01345', 'W01347', 'W01348',
       'W01349', 'W01350', 'W01352', 'W01353', 'W01354', 'W01356',
       'W01358', 'W01361', 'W01363', 'W01368', 'W01369', 'W01370',
       'W01371', 'W01372', 'W01374', 'W01377', 'W01379', 'W01380',
       'W01381', 'W01382', 'W01383', 'W01384', 'W01386', 'W01387',
       'W01388', 'W01390', 'W01392', 'W01394', 'W01395', 'W013

<hr>

What are the top 10 start/end stations:<br />
[ ] on a weekday by registered/casual<br />
[ ] on a weekend by registered/casual<br />
[ ] on a holiday by registered/casual => need to insert holidays<br />
[ ] on a weekday during rush hours (5:00 AM to 9:59 AM and 3:00 PM to 7:59 PM) by registered<br />


In [154]:
# Top ten station combination of registered members during the weekday
weekday_trips = bike_trips.loc[(bike_trips['weekday'] < 5) & (bike_trips['member_type']=='Member')]
top_10 = weekday_trips.groupby(['start_station','end_station']).size().sort_values(ascending=False)
top_10.head(10)

start_station                                        end_station                                        
Eastern Market Metro / Pennsylvania Ave & 7th St SE  Lincoln Park / 13th & East Capitol St NE               8146
Lincoln Park / 13th & East Capitol St NE             Eastern Market Metro / Pennsylvania Ave & 7th St SE    6682
Adams Mill & Columbia Rd NW                          Calvert & Biltmore St NW                               6343
Calvert & Biltmore St NW                             Adams Mill & Columbia Rd NW                            5936
13th & D St NE                                       Columbus Circle / Union Station                        4662
Massachusetts Ave & Dupont Circle NW                 15th & P St NW                                         4378
Columbus Circle / Union Station                      13th & D St NE                                         4278
Crystal City Metro / 18th & Bell St                  27th & Crystal Dr                                  

In [155]:
# Top ten station combination of casual members during the weekday
weekday_trips = bike_trips.loc[(bike_trips['weekday'] < 5) & (bike_trips['member_type']=='Casual')]
top_10 = weekday_trips.groupby(['start_station','end_station']).size().sort_values(ascending=False)
top_10.head(10)

start_station                                          end_station                                          
Jefferson Dr & 14th St SW                              Jefferson Dr & 14th St SW                                2409
Smithsonian-National Mall / Jefferson Dr & 12th St SW  Smithsonian-National Mall / Jefferson Dr & 12th St SW    1900
USDA / 12th & Independence Ave SW                      USDA / 12th & Independence Ave SW                        1865
19th St & Constitution Ave NW                          19th St & Constitution Ave NW                            1740
Ohio Dr & West Basin Dr SW / MLK & FDR Memorials       Ohio Dr & West Basin Dr SW / MLK & FDR Memorials         1550
C & O Canal & Wisconsin Ave NW                         C & O Canal & Wisconsin Ave NW                           1084
Georgetown Harbor / 30th St NW                         Georgetown Harbor / 30th St NW                           1047
New York Ave & 15th St NW                              New York Ave & 15

In [156]:
# Top ten station combination of registered members during the weekend
weekday_trips = bike_trips.loc[(bike_trips['weekday'] >= 5) & (bike_trips['member_type']=='Member')]
top_10 = weekday_trips.groupby(['start_station','end_station']).size().sort_values(ascending=False)
top_10.head(10)

start_station                                        end_station                                        
Adams Mill & Columbia Rd NW                          Calvert & Biltmore St NW                               2203
Calvert & Biltmore St NW                             Adams Mill & Columbia Rd NW                            2089
New Hampshire Ave & T St NW                          Massachusetts Ave & Dupont Circle NW                   1804
Lincoln Park / 13th & East Capitol St NE             Eastern Market Metro / Pennsylvania Ave & 7th St SE    1673
Massachusetts Ave & Dupont Circle NW                 15th & P St NW                                         1650
Eastern Market Metro / Pennsylvania Ave & 7th St SE  Lincoln Park / 13th & East Capitol St NE               1599
15th & P St NW                                       New Hampshire Ave & T St NW                            1584
                                                     Massachusetts Ave & Dupont Circle NW               

In [157]:
# Top ten station combination of casual members during the weekend
weekday_trips = bike_trips.loc[(bike_trips['weekday'] >= 5) & (bike_trips['member_type']=='Casual')]
top_10 = weekday_trips.groupby(['start_station','end_station']).size().sort_values(ascending=False)
top_10.head(10)

start_station                                          end_station                                          
USDA / 12th & Independence Ave SW                      USDA / 12th & Independence Ave SW                        2152
19th St & Constitution Ave NW                          19th St & Constitution Ave NW                            1982
Jefferson Dr & 14th St SW                              Jefferson Dr & 14th St SW                                1749
C & O Canal & Wisconsin Ave NW                         C & O Canal & Wisconsin Ave NW                           1404
Ohio Dr & West Basin Dr SW / MLK & FDR Memorials       Ohio Dr & West Basin Dr SW / MLK & FDR Memorials         1311
Smithsonian-National Mall / Jefferson Dr & 12th St SW  Smithsonian-National Mall / Jefferson Dr & 12th St SW    1287
Georgetown Harbor / 30th St NW                         Georgetown Harbor / 30th St NW                           1271
Massachusetts Ave & Dupont Circle NW                   Massachusetts Ave

In [158]:
# Top ten station combination of registered members during morning weekday rushhour
weekday_trips = bike_trips.loc[(bike_trips['weekday'] < 5) & (bike_trips['member_type']=='Member') & 
                               (bike_trips['start_date'] >= '2011-1-1 5:00:00') & (bike_trips['start_date'] < '2012-12-31 10:00:00')]
top_10 = weekday_trips.groupby(['start_station','end_station']).size().sort_values(ascending=False)
top_10.head(10)

start_station                                        end_station                                        
Eastern Market Metro / Pennsylvania Ave & 7th St SE  Lincoln Park / 13th & East Capitol St NE               8132
Lincoln Park / 13th & East Capitol St NE             Eastern Market Metro / Pennsylvania Ave & 7th St SE    6674
Adams Mill & Columbia Rd NW                          Calvert & Biltmore St NW                               6341
Calvert & Biltmore St NW                             Adams Mill & Columbia Rd NW                            5932
13th & D St NE                                       Columbus Circle / Union Station                        4659
Massachusetts Ave & Dupont Circle NW                 15th & P St NW                                         4374
Columbus Circle / Union Station                      13th & D St NE                                         4277
Crystal City Metro / 18th & Bell St                  27th & Crystal Dr                                  

In [159]:
# Top ten station combination of registered members during evening weekday rushhour
weekday_trips = bike_trips.loc[(bike_trips['weekday'] < 5) & (bike_trips['member_type']=='Member') & 
                               (bike_trips['start_date'] >= '2011-1-1 15:00:00') & (bike_trips['start_date'] < '2012-12-31 20:00:00')]
top_10 = weekday_trips.groupby(['start_station','end_station']).size().sort_values(ascending=False)
top_10.head(10)

start_station                                        end_station                                        
Eastern Market Metro / Pennsylvania Ave & 7th St SE  Lincoln Park / 13th & East Capitol St NE               8146
Lincoln Park / 13th & East Capitol St NE             Eastern Market Metro / Pennsylvania Ave & 7th St SE    6680
Adams Mill & Columbia Rd NW                          Calvert & Biltmore St NW                               6343
Calvert & Biltmore St NW                             Adams Mill & Columbia Rd NW                            5936
13th & D St NE                                       Columbus Circle / Union Station                        4662
Massachusetts Ave & Dupont Circle NW                 15th & P St NW                                         4377
Columbus Circle / Union Station                      13th & D St NE                                         4278
Crystal City Metro / 18th & Bell St                  27th & Crystal Dr                                  

In [160]:
bike_trips.dtypes

duration_sec                     int64
start_date              datetime64[ns]
end_date                datetime64[ns]
start_station_number             int64
start_station                   object
end_station_number               int64
end_station                     object
bike_number                     object
member_type                   category
year                          category
date                            object
hour                            object
date_hour               datetime64[ns]
weekday                          int64
dtype: object

In [161]:
weekday_trips.head(4)

,duration_sec,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,year,date,hour,date_hour,weekday
1740,183,2011-01-03 00:01:08,2011-01-03 00:04:12,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31606,Potomac & Pennsylvania Ave SE,W00518,Member,2011,2011-01-03,00:00:00,2011-01-03,0
1741,294,2011-01-03 00:08:44,2011-01-03 00:13:39,31613,Eastern Market Metro / Pennsylvania Ave & 7th ...,31619,Lincoln Park / 13th & East Capitol St NE,W00922,Member,2011,2011-01-03,00:00:00,2011-01-03,0
1742,577,2011-01-03 00:26:37,2011-01-03 00:36:15,31201,15th & P St NW,31602,Park Rd & Holmead Pl NW,W00240,Member,2011,2011-01-03,00:00:00,2011-01-03,0
1743,584,2011-01-03 00:45:42,2011-01-03 00:55:26,31110,20th St & Florida Ave NW,31602,Park Rd & Holmead Pl NW,W00155,Member,2011,2011-01-03,00:00:00,2011-01-03,0


In [162]:

bike_trips = bike_trips.drop('date', 1)
bike_trips = bike_trips.drop('date_hour', 1)
bike_trips = bike_trips.drop('year', 1)
bike_trips = bike_trips.drop('hour', 1)

bike_trips["year"]=pd.DatetimeIndex(bike_trips["start_date"]).year
bike_trips["month"]=pd.DatetimeIndex(bike_trips["start_date"]).month
bike_trips["day"]=pd.DatetimeIndex(bike_trips["start_date"]).day
bike_trips["hour"]=pd.DatetimeIndex(bike_trips["start_date"]).hour
bike_trips["weekday"]=pd.DatetimeIndex(bike_trips["start_date"]).weekday


/tmp/ipykernel_5505/1372878214.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  bike_trips = bike_trips.drop('date', 1)
/tmp/ipykernel_5505/1372878214.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  bike_trips = bike_trips.drop('date_hour', 1)
/tmp/ipykernel_5505/1372878214.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  bike_trips = bike_trips.drop('year', 1)
/tmp/ipykernel_5505/1372878214.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  bike_trips = bike_trips.drop('hour', 1)


In [163]:
bike_trips.head(2)

,duration_sec,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,weekday,year,month,day,hour
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,5th & F St NW,31620,5th & F St NW,W00247,Member,5,2011,1,1,0
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,14th & Harvard St NW,31101,14th & V St NW,W00675,Casual,5,2011,1,1,0


In [164]:
bike_trips['workingday'] = bike_trips['weekday'] < 5

In [165]:
# ?? import holidays

: 